# KBO 사이트 데이터 정적크롤링 코드 (2025)

In [2]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd

result = []

KBO_url = 'https://www.koreabaseball.com/Record/Crowd/GraphDaily.aspx'

print("크롤링 시작:", KBO_url)
html = urllib.request.urlopen(KBO_url)
soupKBO = BeautifulSoup(html, 'html.parser')

# 테이블 본문 추출
tag_tbody = soupKBO.find('tbody')

# ✅ 잘못된 부분 수정: find_ → find_all
for store in tag_tbody.find_all('tr'):
    store_td = store.find_all('td')
    if len(store_td) < 6:
        continue  # 불완전한 행은 스킵

    store_date = store_td[0].text.strip()
    store_day = store_td[1].text.strip()
    store_home = store_td[2].text.strip()
    store_go = store_td[3].text.strip()
    store_address = store_td[4].text.strip()
    store_population = store_td[5].text.strip().replace(",", "")

    result.append([store_date, store_day, store_home, store_go, store_address, int(store_population)])

# 데이터프레임 생성
KBO_tbl = pd.DataFrame(result, columns=('날짜', '요일', '홈', '방문', '구장', '관중수'))

# 저장 (한글 깨짐 방지 위해 cp949 또는 utf-8-sig)
KBO_tbl.to_csv("C:/ex/Project_KBO.csv", encoding="cp949", mode="w", index=False)

print("✅ 엑셀(CSV) 저장 완료!")

크롤링 시작: https://www.koreabaseball.com/Record/Crowd/GraphDaily.aspx
✅ 엑셀(CSV) 저장 완료!


정적크롤링 한계: KBO 사이트의 년도 구분이 페이지가 아닌 클릭 방식으로 이루어져 동적크로링을 해야할 것으로 판단. 

# KBO 사이트 데이터 동적크롤링 코드 (2023-2025)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

# 🔧 크롬 드라이버 경로 설정
from selenium.webdriver.chrome.service import Service
chrome_path = "C:/Users/USER/Downloads/chromedriver-win64/chromedriver-win64/chromedriver.exe"
service = Service(executable_path=chrome_path)
options = webdriver.ChromeOptions()
options.add_argument('--headless')  # 창 숨기기

driver = webdriver.Chrome(service=service, options=options)
url = "https://www.koreabaseball.com/Record/Crowd/GraphDaily.aspx"
driver.get(url)
time.sleep(10)

# 🔁 수집할 연도 목록
years = ['2023', '2024', '2025']
all_data = []

for year in years:
    print(f"🔍 {year}년 데이터 수집 중...")

    # 📌 연도 선택 드롭다운 제어
    select = Select(driver.find_element(By.ID, "cphContents_cphContents_cphContents_ddlSeason"))
    select.select_by_visible_text(year)
    time.sleep(10)  # 데이터 다시 로딩될 때까지 대기

    # 📥 테이블 크롤링
    rows = driver.find_elements(By.CSS_SELECTOR, "#cphContents_cphContents_cphContents_udpRecord > table > tbody > tr")
    
    for row in rows:
        cols = row.find_elements(By.TAG_NAME, "td")
        if len(cols) == 6:
            date = cols[0].text.strip()
            day = cols[1].text.strip()
            home = cols[2].text.strip()
            away = cols[3].text.strip()
            stadium = cols[4].text.strip()
            audience = cols[5].text.strip().replace(",", "")
            all_data.append([year, date, day, home, away, stadium, int(audience)])

# ✅ 브라우저 종료
driver.quit()

# ✅ DataFrame으로 저장
df = pd.DataFrame(all_data, columns=["연도", "날짜", "요일", "홈", "방문", "구장", "관중수"])
df.to_excel("KBO_관중수_2023_2025.xlsx", index=False)

print("✅ 전체 연도 데이터 저장 완료!")

🔍 2023년 데이터 수집 중...
🔍 2024년 데이터 수집 중...
🔍 2025년 데이터 수집 중...
✅ 전체 연도 데이터 저장 완료!
